In [20]:
import tsplib95


import urllib.request
import gzip
import shutil

# Descargar el archivo comprimido
file = "swiss42.tsp"
urllib.request.urlretrieve(
    "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz",
    file + ".gz"
)

# Descomprimir usando Python
with gzip.open(file + ".gz", 'rb') as f_in:
    with open(file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Ahora sí puedes cargar el archivo
problem = tsplib95.load(file)
print("Nombre del problema:", problem.name)

Nodos = list(problem.get_nodes())
Aristas = list(problem.get_edges())

import random
# Funciones básicas
# Se genera una solución aleatoria con comienzo en el nodo 0
def crear_solucion(Nodos):
    solucion = [Nodos[0]]
    for n in Nodos[1:]:
        solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
    return solucion

# Devuelve la distancia entre dos nodos
def distancia(a, b, problem):
    return problem.get_weight(a, b)

# Devuelve la distancia total de una trayectoria/solución
def distancia_total(solucion, problem):
    distancia_total = 0
    for i in range(len(solucion) - 1):
        distancia_total += distancia(solucion[i], solucion[i + 1], problem)
    distancia_total += distancia(solucion[-1], solucion[0], problem)
    return distancia_total

# Uso
sol_temporal = crear_solucion(Nodos)

Nombre del problema: swiss42


In [21]:
###############################################################################
# SIMULATED ANNEALING
###############################################################################
#Generador de 1 solucion vecina 2-opt 100% aleatoria (intercambiar 2 nodos)
#Mejorable eligiendo otra forma de elegir una vecina.
def genera_vecina_aleatorio(solucion):
#Se eligen dos nodos aleatoriamente
    i,j = sorted(random.sample( range(1,len(solucion)) , 2))
    #Devuelve una nueva solución pero intercambiando los dos nodos elegidos al␣ azar
    return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
    #Funcion de probabilidad para aceptar peores soluciones
def probabilidad(T,d):
    if random.random() < math.exp( -1*d / T) :
        return True
    else:
        return False
#Funcion de descenso de temperatura
def bajar_temperatura(T):
    return T*0.99

In [22]:
import math

def recocido_simulado(problem, TEMPERATURA):
    #problem = datos del problema
    #T = Temperatura

    solucion_referencia = crear_solucion(Nodos)
    distancia_referencia = distancia_total(solucion_referencia, problem)

    mejor_solucion = []            #x* del seudocodigo
    mejor_distancia = 10e100      #F* del seudocodigo

    N = 0
    while TEMPERATURA > .0001:
        N += 1

        #Genera una solución vecina
        vecina = genera_vecina_aleatorio(solucion_referencia)

        #Calcula su valor (distancia)
        distancia_vecina = distancia_total(vecina, problem)

        #Si es la mejor solución de todas se guarda (¡siempre!)
        if distancia_vecina < mejor_distancia:
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina

        #Si la nueva vecina es mejor se cambia
        #Si es peor se cambia según una probabilidad que depende de T y
        #delta(distancia_referencia - distancia_vecina)
        if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA,
           abs(distancia_referencia - distancia_vecina)):
            
            #solucion_referencia = copy.deepcopy(vecina)
            solucion_referencia = vecina
            distancia_referencia = distancia_vecina

        #Bajamos la temperatura
        TEMPERATURA = bajar_temperatura(TEMPERATURA)

    print("La mejor solución encontrada es ", end="")
    print(mejor_solucion)
    print("con una distancia total de ", end="")
    print(mejor_distancia)
    return mejor_solucion



sol = recocido_simulado(problem, 10000000)

La mejor solución encontrada es [0, 1, 4, 3, 27, 2, 26, 19, 14, 15, 16, 13, 18, 12, 11, 25, 40, 24, 21, 39, 9, 34, 20, 33, 35, 36, 7, 6, 5, 37, 17, 31, 32, 8, 23, 41, 10, 28, 29, 22, 38, 30]
con una distancia total de 1905
